In [19]:
!pip install ortools
from ortools.sat.python import cp_model
import pandas as pd
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [23]:
nurses

,Nurse,Team
0,0,A
1,1,A
2,2,B
3,3,B
4,4,B
5,5,C
6,6,C
7,7,D
8,8,D
9,9,E


In [22]:
# assumes first day is a Monday
nurses = pd.read_csv("gdrive/My Drive/Nurses.csv")
a_nurses = nurses.loc[nurses["Team"] == "A"]["Nurse"].tolist()
num_nurses = len(nurses["Nurse"])
num_shifts = 3
num_days = 56
all_nurses = range(num_nurses)
all_shifts = range(num_shifts)
all_days = range(num_days)
# for just weekends, see below line
# all_weekends = range(2*(num_days//7) + 1) if num_days % 7 == 6 else range(2*(num_days//7))

model = cp_model.CpModel()

# assigns a boolean variable to each of the decisions
shifts = {}
for n in all_nurses:
    for d in all_days:
        for s in all_shifts:
            shifts[(n, d, s)] = model.NewBoolVar("shift_n%sd%is%i" % (n, d, s))

# each shift per day can only have 1 nurse assigned to it
for d in all_days:
    for s in all_shifts:
        model.Add(sum(shifts[(n, d, s)] for n in all_nurses) == 1)

# all nurses can only have one shift or less per day
for n in all_nurses:
    for d in all_days:
        model.Add(sum(shifts[(n, d, s)] for s in all_shifts) <= 1)

# ensure that a nurse does not work shift 0 or 1 for two days consecutively
for n in all_nurses:
    for d in all_days[:-1]:
        model.Add(shifts[(n, d, 0)] + shifts[(n, d + 1, 0)] <= 1)
        model.Add(shifts[(n, d, 0)] + shifts[(n, d + 1, 1)] <= 1)
        model.Add(shifts[(n, d, 1)] + shifts[(n, d + 1, 0)] <= 1)
        model.Add(shifts[(n, d, 1)] + shifts[(n, d + 1, 1)] <= 1)

# combination fo two previous constraints
# for n in all_nurses:
#     for d in all_days[:-1]:
#         model.Add(sum(shifts[(n, d, s)] for s in all_shifts[:2]) +
#                   sum(shifts[(n, d+1, s)] for s in all_shifts[:2]) <= 1)

# ensure that if a nurse works shift 2 on a Monday, it will work shift 2 on Tuesday and Wednesday
for n in all_nurses:
    for d in all_days[::7]:
        if d + 1 < num_days:
          model.Add(shifts[(n, d+1, 2)] == 1).OnlyEnforceIf(shifts[(n, d, 2)])
        if d + 2 < num_days:
          model.Add(shifts[(n, d+2, 2)] == 1).OnlyEnforceIf(shifts[(n, d, 2)])

# ensure that no nurse works more than 4 shifts per week
for n in all_nurses:
    for w in all_days[::7]:
        model.Add(sum(shifts[(n, d, s)] for d in all_days[w:w+7] for s in all_shifts) <= 4)

# ensures that all nurses apart of A team do not work any shift 0s
for d in all_days:
    model.Add(sum(shifts[(n, d, 0)] for n in a_nurses) == 0)

# ensures that no more than 1 nurse from each team is working each day
for d in all_days:
    for team in ["A", "B", "C", "D", "E"]:
        working_nurses = []
        for n in nurses.loc[nurses["Team"] == team]["Nurse"].tolist():
             nurse_working = model.NewBoolVar(f"nurse_{n}_working_day_{d}")
             model.AddMaxEquality(nurse_working, [shifts[(n, d, s)] for s in all_shifts])
             working_nurses.append(nurse_working)
        model.Add(sum(working_nurses) <= 1)

# Try to distribute the shifts evenly, so that each nurse works
# min_shifts_per_nurse shifts. If this is not possible, because the total
# number of shifts is not divisible by the number of nurses, some nurses will
# be assigned one more shift.

min_shifts_per_nurse = (num_shifts * len(all_days)) // num_nurses
if num_shifts * len(all_days) % num_nurses == 0:
    max_shifts_per_nurse = min_shifts_per_nurse
else:
    max_shifts_per_nurse = min_shifts_per_nurse + 1
for n in all_nurses:
    num_shifts_worked = sum(shifts[(n, d, s)] for d in all_days for s in all_shifts)
    model.Add(num_shifts_worked >= min_shifts_per_nurse)
    model.Add(num_shifts_worked <= max_shifts_per_nurse)

solver = cp_model.CpSolver()
status = solver.Solve(model)

if status == cp_model.OPTIMAL:
    rota_dict = {(n, d, s) for (n, d, s) in shifts if solver.Value(shifts[(n, d, s)]) == 1}
    Solution = pd.DataFrame(rota_dict, columns = ["Nurse", "Day", "Shift"]).sort_values(by="Day")
    Rota = Solution.pivot(index="Day", columns = "Shift", values = "Nurse")
else:
    print("ERROR: No Solution")

print(Solution)

     Nurse  Day  Shift
118      1    0      2
2        6    0      0
58       9    0      1
76       5    1      0
70       2    1      1
..     ...  ...    ...
46       8   54      0
132      6   54      1
53       4   55      0
120      1   55      2
14       9   55      1

[168 rows x 3 columns]


In [21]:
# clean this up and decide what I want to produce

# Summarises the data to show how many shifts in total a nurse has on the weekends and weekdays or Saturday/Sunday
for i in all_nurses:
  nurse_shift = Solution.loc[Solution.Nurse == i]
  #nurse_shift.loc[:, "Day"] = nurse_shift["Day"].apply(lambda x: "Sunday" if x % 7 == 6 else ("Saturday" if x % 7 == 5 else "Weekday"))
  nurse_shift.loc[:, "Day"] = nurse_shift["Day"].apply(lambda x: "Weekend" if x % 7 == 6 or x % 7 == 5 else "Weekday")
  #num_sun = sum(nurse_shift["Day"] == "Sunday")
  #num_sat = sum(nurse_shift["Day"] == "Saturday")
  num_day = sum(nurse_shift["Day"] == "Weekend")
  num_end = sum(nurse_shift["Day"] == "Weekday")
  print(f"Nurse {i} has {num_day} shift(s) on during Weekdays")
  print(f"Nurse {i} has {num_end} shift(s) on during Weekends")
  #print(f"Nurse {i} has {num_sun} shift(s) on Sunday")
  #print(f"Nurse {i} has {num_sat} shift(s) on Saturday")
  # print(nurse_shift)

Nurse 0 has 3 shift(s) on during Weekdays
Nurse 0 has 14 shift(s) on during Weekends
Nurse 1 has 5 shift(s) on during Weekdays
Nurse 1 has 11 shift(s) on during Weekends
Nurse 2 has 5 shift(s) on during Weekdays
Nurse 2 has 12 shift(s) on during Weekends
Nurse 3 has 4 shift(s) on during Weekdays
Nurse 3 has 13 shift(s) on during Weekends
Nurse 4 has 5 shift(s) on during Weekdays
Nurse 4 has 12 shift(s) on during Weekends
Nurse 5 has 2 shift(s) on during Weekdays
Nurse 5 has 15 shift(s) on during Weekends
Nurse 6 has 6 shift(s) on during Weekdays
Nurse 6 has 11 shift(s) on during Weekends
Nurse 7 has 6 shift(s) on during Weekdays
Nurse 7 has 11 shift(s) on during Weekends
Nurse 8 has 5 shift(s) on during Weekdays
Nurse 8 has 11 shift(s) on during Weekends
Nurse 9 has 7 shift(s) on during Weekdays
Nurse 9 has 10 shift(s) on during Weekends


<ipython-input-21-7512ae649fb6>:7: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '['Weekday' 'Weekday' 'Weekday' 'Weekday' 'Weekend' 'Weekday' 'Weekday'
 'Weekday' 'Weekday' 'Weekday' 'Weekday' 'Weekend' 'Weekday' 'Weekend'
 'Weekday' 'Weekday' 'Weekday']' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  nurse_shift.loc[:, "Day"] = nurse_shift["Day"].apply(lambda x: "Weekend" if x % 7 == 6 or x % 7 == 5 else "Weekday")
<ipython-input-21-7512ae649fb6>:7: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '['Weekday' 'Weekday' 'Weekday' 'Weekday' 'Weekday' 'Weekday' 'Weekday'
 'Weekend' 'Weekday' 'Weekend' 'Weekday' 'Weekend' 'Weekday' 'Weekday'
 'Weekend' 'Weekend']' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  nurse_shift.loc[:, "Day"] = nurse_shift["Day"].apply(lambda